# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 6 2022 2:57PM,245773,20,8475523,"ACI Healthcare USA, Inc.",Released
1,Sep 6 2022 2:57PM,245773,20,8475526,"ACI Healthcare USA, Inc.",Released
2,Sep 6 2022 2:57PM,245773,20,8475528,"ACI Healthcare USA, Inc.",Released
3,Sep 6 2022 2:57PM,245773,20,8475529,"ACI Healthcare USA, Inc.",Released
4,Sep 6 2022 2:57PM,245773,20,8475530,"ACI Healthcare USA, Inc.",Released
5,Sep 6 2022 2:57PM,245773,20,8475531,"ACI Healthcare USA, Inc.",Released
6,Sep 6 2022 2:57PM,245773,20,8475532,"ACI Healthcare USA, Inc.",Released
7,Sep 6 2022 2:57PM,245773,20,8475533,"ACI Healthcare USA, Inc.",Released
8,Sep 6 2022 2:57PM,245773,20,8475536,"ACI Healthcare USA, Inc.",Released
9,Sep 6 2022 2:57PM,245773,20,8475537,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,245767,Executing,1
37,245768,Released,18
38,245769,Released,6
39,245772,Released,45
40,245773,Released,36


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
245767,NaN,NaN,1.0,NaN
245768,NaN,NaN,NaN,18.0
245769,NaN,NaN,NaN,6.0
245772,NaN,NaN,NaN,45.0
245773,NaN,NaN,NaN,36.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
245647,0.0,0.0,0.0,1.0
245654,0.0,0.0,1.0,0.0
245659,0.0,0.0,0.0,1.0
245689,0.0,0.0,6.0,1.0
245704,0.0,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
245647,0.0,0,0,1
245654,0.0,0,1,0
245659,0.0,0,0,1
245689,0.0,0,6,1
245704,0.0,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,245647,0.0,0,0,1
1,245654,0.0,0,1,0
2,245659,0.0,0,0,1
3,245689,0.0,0,6,1
4,245704,0.0,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,245647,0.0,,,1
1,245654,0.0,,1,
2,245659,0.0,,,1
3,245689,0.0,,6,1
4,245704,0.0,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 6 2022 2:57PM,245773,20,"ACI Healthcare USA, Inc."
36,Sep 6 2022 2:46PM,245772,10,Emerginnova
81,Sep 6 2022 2:33PM,245769,10,"Methapharm, Inc."
87,Sep 6 2022 2:33PM,245768,10,"Methapharm, Inc."
105,Sep 6 2022 2:27PM,245767,10,"Methapharm, Inc."
106,Sep 6 2022 2:24PM,245756,10,ISDIN Corporation
175,Sep 6 2022 2:20PM,245757,10,ISDIN Corporation
234,Sep 6 2022 2:19PM,245766,10,"Methapharm, Inc."
239,Sep 6 2022 2:18PM,245755,10,ISDIN Corporation
296,Sep 6 2022 2:18PM,245765,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Sep 6 2022 2:57PM,245773,20,"ACI Healthcare USA, Inc.",0.0,,,36
1,Sep 6 2022 2:46PM,245772,10,Emerginnova,0.0,,,45
2,Sep 6 2022 2:33PM,245769,10,"Methapharm, Inc.",0.0,,,6
3,Sep 6 2022 2:33PM,245768,10,"Methapharm, Inc.",0.0,,,18
4,Sep 6 2022 2:27PM,245767,10,"Methapharm, Inc.",0.0,,1,
5,Sep 6 2022 2:24PM,245756,10,ISDIN Corporation,0.0,,,69
6,Sep 6 2022 2:20PM,245757,10,ISDIN Corporation,0.0,,,59
7,Sep 6 2022 2:19PM,245766,10,"Methapharm, Inc.",0.0,,,5
8,Sep 6 2022 2:18PM,245755,10,ISDIN Corporation,0.0,,,57
9,Sep 6 2022 2:18PM,245765,10,Eywa Pharma Inc.,0.0,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 6 2022 2:57PM,245773,20,"ACI Healthcare USA, Inc.",36,,
1,Sep 6 2022 2:46PM,245772,10,Emerginnova,45,,
2,Sep 6 2022 2:33PM,245769,10,"Methapharm, Inc.",6,,
3,Sep 6 2022 2:33PM,245768,10,"Methapharm, Inc.",18,,
4,Sep 6 2022 2:27PM,245767,10,"Methapharm, Inc.",,1,
5,Sep 6 2022 2:24PM,245756,10,ISDIN Corporation,69,,
6,Sep 6 2022 2:20PM,245757,10,ISDIN Corporation,59,,
7,Sep 6 2022 2:19PM,245766,10,"Methapharm, Inc.",5,,
8,Sep 6 2022 2:18PM,245755,10,ISDIN Corporation,57,,
9,Sep 6 2022 2:18PM,245765,10,Eywa Pharma Inc.,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 6 2022 2:57PM,245773,20,"ACI Healthcare USA, Inc.",36,,
1,Sep 6 2022 2:46PM,245772,10,Emerginnova,45,,
2,Sep 6 2022 2:33PM,245769,10,"Methapharm, Inc.",6,,
3,Sep 6 2022 2:33PM,245768,10,"Methapharm, Inc.",18,,
4,Sep 6 2022 2:27PM,245767,10,"Methapharm, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 6 2022 2:57PM,245773,20,"ACI Healthcare USA, Inc.",36.0,NaN,NaN
1,Sep 6 2022 2:46PM,245772,10,Emerginnova,45.0,NaN,NaN
2,Sep 6 2022 2:33PM,245769,10,"Methapharm, Inc.",6.0,NaN,NaN
3,Sep 6 2022 2:33PM,245768,10,"Methapharm, Inc.",18.0,NaN,NaN
4,Sep 6 2022 2:27PM,245767,10,"Methapharm, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 6 2022 2:57PM,245773,20,"ACI Healthcare USA, Inc.",36.0,0.0,0.0
1,Sep 6 2022 2:46PM,245772,10,Emerginnova,45.0,0.0,0.0
2,Sep 6 2022 2:33PM,245769,10,"Methapharm, Inc.",6.0,0.0,0.0
3,Sep 6 2022 2:33PM,245768,10,"Methapharm, Inc.",18.0,0.0,0.0
4,Sep 6 2022 2:27PM,245767,10,"Methapharm, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4669371,448.0,56.0,0.0
12,491463,6.0,1.0,0.0
15,245724,14.0,7.0,0.0
16,491463,4.0,0.0,0.0
19,491480,7.0,20.0,2.0
20,737224,38.0,6.0,0.0
21,1228404,4.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4669371,448.0,56.0,0.0
1,12,491463,6.0,1.0,0.0
2,15,245724,14.0,7.0,0.0
3,16,491463,4.0,0.0,0.0
4,19,491480,7.0,20.0,2.0
5,20,737224,38.0,6.0,0.0
6,21,1228404,4.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,448.0,56.0,0.0
1,12,6.0,1.0,0.0
2,15,14.0,7.0,0.0
3,16,4.0,0.0,0.0
4,19,7.0,20.0,2.0
5,20,38.0,6.0,0.0
6,21,4.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,448.0
1,12,Released,6.0
2,15,Released,14.0
3,16,Released,4.0
4,19,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,2.0,0.0,0.0
Executing,56.0,1.0,7.0,0.0,20.0,6.0,1.0
Released,448.0,6.0,14.0,4.0,7.0,38.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,Executing,56.0,1.0,7.0,0.0,20.0,6.0,1.0
2,Released,448.0,6.0,14.0,4.0,7.0,38.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,Executing,56.0,1.0,7.0,0.0,20.0,6.0,1.0
2,Released,448.0,6.0,14.0,4.0,7.0,38.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()